In [47]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.utils.data

from sklearn.mixture import GaussianMixture 

import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['date'] = pd.to_datetime(data.date)
    print('Days: ',len(set(data.date)))
    return data

In [49]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table,table.index.values

In [50]:
def getMatrixData():
    file = dataDir + dataFile
    dataRaw = loadData(file)
    dataTs,dates = getTimeSeries(dataRaw)
    matrix = dataTs.values
    print('Matrix Shape: ',matrix.shape)
    return matrix,dates

In [51]:
dataDir = '/home/urwa/Documents/Projects/AnomalyDetection/Pipeline/data/'
dataFile = '20190402_TaipeiEdgesDatewise.csv'
events_data =dataDir+'TaipeiEvents.csv'

In [52]:
#matrix = np.stack((dataOut.values, dataIn.values),-1)
matrix,dates = getMatrixData()
matrix = matrix.astype(float)
matrix.shape

Raw shape:  (7374816, 5)
Days:  638
Matrix Shape:  (638, 11664)


(638, 11664)

In [53]:
# for i in range(matrix.shape[1]):
#     matrix[:, i] = (matrix[:, i] - matrix[:, i].min()) / (matrix[:, i].max() - matrix[:, i].min())

In [54]:
for i in range(matrix.shape[0]):
    matrix[i, :] = (matrix[i, :] - matrix[i, :].min()) / (matrix[i, :].max() - matrix[i, :].min())

In [55]:
# for i in range(matrix.shape[1]):
#     for j in range(matrix.shape[2]):
#         matrix[:, i,j] = (matrix[:, i,j] - np.mean(matrix[:, i,j])) / (np.std(matrix[:, i,j]))

In [56]:
#stations = list(set(dataRaw.start_id))
#n= len(stations)
# edge_index = [[a//n,a%n] for a in range(n*n)]
# edge_index = torch.tensor(edge_index, dtype=torch.long)

In [57]:
#dates = list(dataOut.index)
DOW = list(pd.to_datetime(dates).dayofweek)
DOW = ((np.array(DOW) == 5) | (np.array(DOW) == 6)).astype(int)
DOW[:10]

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [58]:
# dataList = []

# for i in range(len(DOW)):
#     x = torch.tensor(matrix[i], dtype=torch.float)
#     y = torch.tensor(np.array([DOW[i]]), dtype=torch.long)
#     data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y)
#     dataList.append(data)

# dataList[0]

In [59]:
class autoencoder(nn.Module):
    def __init__(self,inputD,encoding_dim):
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential()
        
        self.encoder.add_module("enc_0", nn.Linear(inputD,encoding_dim[0]))
        self.encoder.add_module("relu_0", nn.ReLU())
          
        for l in range(1,len(encoding_dim)):
            self.encoder.add_module("enc_"+str(l), nn.Linear(encoding_dim[l-1],encoding_dim[l]))
            self.encoder.add_module("encrelu_"+str(l), nn.ReLU())
                                    
        self.decoder = nn.Sequential()
        
        for l in range(len(encoding_dim)-1,0,-1):
            self.decoder.add_module("dec_"+str(l), nn.Linear(encoding_dim[l],encoding_dim[l-1]))
            self.decoder.add_module("decrelu_"+str(l), nn.ReLU())
            
        self.decoder.add_module("dec_0", nn.Linear(encoding_dim[0],inputD))
        self.decoder.add_module("decrelu_0", nn.Sigmoid())
        
        self.encoder.apply(self.init_weights)
        self.decoder.apply(self.init_weights)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def init_weights(self,m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def representation(self, x):
        x = self.encoder(x)
        return x

In [60]:
matrixtensor = torch.tensor(matrix).float()

In [61]:
input_dim = matrix.shape[1]

In [62]:
encoding_dim = [500,100,20]

learning_rate=0.001
batch_size = 100
num_epochs = 100

In [63]:
data_tensor = torch.utils.data.TensorDataset(matrixtensor, matrixtensor) 
dataloader = torch.utils.data.DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)

In [64]:
model = autoencoder(input_dim,encoding_dim).cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [65]:
for epoch in range(num_epochs):
    for data in dataloader:
        X, _ = data
        X = X.cuda()
        # ===================forward=====================
        output = model(X)
        loss = criterion(output, X)
        MSE_loss = nn.MSELoss()(output, X)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
        .format(epoch + 1, num_epochs, loss.item(), MSE_loss.item()))

epoch [1/100], loss:0.4027, MSE_loss:0.0927
epoch [2/100], loss:0.1895, MSE_loss:0.0015
epoch [3/100], loss:0.1294, MSE_loss:0.0035
epoch [4/100], loss:0.1143, MSE_loss:0.0057
epoch [5/100], loss:0.1061, MSE_loss:0.0027
epoch [6/100], loss:0.0999, MSE_loss:0.0011
epoch [7/100], loss:0.0983, MSE_loss:0.0012
epoch [8/100], loss:0.0909, MSE_loss:0.0010
epoch [9/100], loss:0.0872, MSE_loss:0.0004
epoch [10/100], loss:0.0849, MSE_loss:0.0002
epoch [11/100], loss:0.0869, MSE_loss:0.0001
epoch [12/100], loss:0.0874, MSE_loss:0.0001
epoch [13/100], loss:0.0829, MSE_loss:0.0001
epoch [14/100], loss:0.0886, MSE_loss:0.0002
epoch [15/100], loss:0.0878, MSE_loss:0.0002
epoch [16/100], loss:0.0877, MSE_loss:0.0002
epoch [17/100], loss:0.0911, MSE_loss:0.0001
epoch [18/100], loss:0.0843, MSE_loss:0.0002
epoch [19/100], loss:0.0844, MSE_loss:0.0002
epoch [20/100], loss:0.0910, MSE_loss:0.0001
epoch [21/100], loss:0.0906, MSE_loss:0.0001
epoch [22/100], loss:0.0862, MSE_loss:0.0001
epoch [23/100], los

In [66]:
with torch.no_grad():
    representation = model.representation(matrixtensor.cuda()).cpu().numpy()
representation.shape

(638, 20)

In [67]:
# import events data
events_data =dataDir+'TaipeiEvents.csv'
df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)


In [68]:
df_events.head()

,Type,Name,Date
0,National holiday,Republic Day/New Year's Day observed,2017-01-02
1,National holiday,Chinese New Year's Eve,2017-01-27
2,National holiday,Chinese New Year's Day,2017-01-28
3,National holiday,Chinese New Year Holiday 1,2017-01-29
4,National holiday,Chinese New Year Holiday 2,2017-01-30


In [69]:
holidayDates = df_events[df_events.Type == 'National holiday'].Date

In [70]:
holidayDates = [str(d.date()) for d in holidayDates]
dates = pd.to_datetime(dates)
dates = [str(d.date()) for d in dates]

In [71]:
anomalyIndex = [i for i,d in enumerate(dates) if d in holidayDates]
len(anomalyIndex)
indexBool = np.array([i in anomalyIndex for i in list(range(matrix.shape[0]))])

In [72]:
def anomalyDetection(y,pval = 0.2,iterN=5,n_com=1):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=n_com, n_init=100, max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if(sum(rind)==0):
            print('All anomalies in {} iterations'.format(i+1))
            break
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [73]:
def getResults(reducedMatrix,threshHolds,iterN=5,n_com=1):
    results = []
    for th in threshHolds:
        #th = thres/100
        print("Threshhold: ",th)
        outliers = anomalyDetection(reducedMatrix,th,iterN,n_com)

        tpr = sum(outliers & indexBool)/sum(indexBool)
        fpr = sum(outliers & ~indexBool)/sum(~indexBool)
        precision = sum(outliers & indexBool)/sum(outliers)

        F1 = 2 * (precision * tpr) / (precision + tpr)

        res = {'Cat':'Global', 'th':th, 'TPR':tpr, 'FPR':fpr, 'F1':F1, 'Precision':precision}
        results.append(res)

    resDf = pd.DataFrame(results)    
    return resDf

In [74]:
threshHolds = [0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [75]:
Res1 = getResults(representation,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Convergence in 3 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration

In [76]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.057143,0.008210,0.166667,0.034483,0.01
1,Global,0.208333,0.022989,0.263158,0.172414,0.03
2,Global,0.185185,0.032841,0.200000,0.172414,0.04
3,Global,0.233333,0.039409,0.225806,0.241379,0.05
4,Global,0.268657,0.047619,0.236842,0.310345,0.06
5,Global,0.246575,0.057471,0.204545,0.310345,0.07
6,Global,0.275000,0.065681,0.215686,0.379310,0.08
7,Global,0.304348,0.080460,0.222222,0.482759,0.10
8,Global,0.243590,0.177340,0.149606,0.655172,0.20
9,Global,0.245455,0.269294,0.141361,0.931034,0.30


In [77]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(matrix)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [78]:
reducedMatrixPCA = pca.transform(matrix)
print(reducedMatrixPCA.shape)

(638, 20)


In [79]:
Res1 = getResults(reducedMatrixPCA,threshHolds,iterN=5,n_com=1)

Threshhold:  0.01
Iteration 1
Iteration 2
Convergence in 2 iterations
Threshhold:  0.03
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.04
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.05
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.06
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.07
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.08
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Convergence in 5 iterations
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Convergence in 4 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration

In [80]:
Res1

,Cat,F1,FPR,Precision,TPR,th
0,Global,0.228571,0.003284,0.666667,0.137931,0.01
1,Global,0.458333,0.013136,0.578947,0.379310,0.03
2,Global,0.444444,0.021346,0.480000,0.413793,0.04
3,Global,0.400000,0.031199,0.387097,0.413793,0.05
4,Global,0.388060,0.041051,0.342105,0.448276,0.06
5,Global,0.383562,0.049261,0.318182,0.482759,0.07
6,Global,0.350000,0.060755,0.274510,0.482759,0.08
7,Global,0.369565,0.075534,0.269841,0.586207,0.10
8,Global,0.294872,0.170772,0.181102,0.793103,0.20
9,Global,0.254545,0.267652,0.146597,0.965517,0.30
